In [1]:
!pip install torchvision torch pytorch-grad-cam


ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam


In [2]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 12.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.3-py3-none-any.whl size=38657 sha256=7dbf07631af01c1f188bb3699a3cd053172d7edc5f9b047cb78805dc49676d8d
  Stored in directory: /root/.cache/pip/wheels/2e/ce/70/fe64f851895eae830b3c63ec7fc464cfa7c81aeb7ad4f68063
Successfully built grad-cam


In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM # this should now work
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Path to your dataset (replace with your actual path in Google Drive)
img_folder = '/content/drive/MyDrive/pocs/'
infected_folder = img_folder + 'infected/'
non_infected_folder = img_folder + 'notinfected/' #Fixed typo: changed 'non_infected' to 'non-infected'


In [6]:
import os
img_paths = []
labels = []

In [7]:
# Assuming you have two folders 'infected' and 'non_infected'
# Collect infected images
for img_name in os.listdir(infected_folder):
    img_paths.append(os.path.join(infected_folder, img_name))
    labels.append(1)  # Label 1 for infected

In [8]:
# Collect non-infected images
for img_name in os.listdir(non_infected_folder):
    img_paths.append(os.path.join(non_infected_folder, img_name))
    labels.append(0)  # Label 0 for non-infected

In [9]:
# Define the MedicalDataset class
class MedicalDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB') # Ensure the image is loaded as RGB
        if self.transform:
            image = self.transform(image)
        return image, label

In [10]:
# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [11]:
# Create dataset and dataloader
dataset = MedicalDataset(img_paths, labels, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [12]:

# Load DenseNet201 pre-trained model
model = models.densenet201(pretrained=True)

# Replace the classifier layer to fit your number of classes
model.classifier = nn.Linear(1920, 2)  # Assuming binary classification (infected vs non-infected)

# Move the model to the appropriate device (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 85.6MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25  # Adjust based on your dataset

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader)}')


In [ ]:
# Apply Grad-CAM
target_layers = [model.features.denseblock4]  # Select the target layer to visualize
cam = GradCAM(model=model, target_layers=target_layers, use_cuda=torch.cuda.is_available())




In [ ]:
# Test the model on a new image
image_path = '/content/drive/MyDrive/pocs/infected/1 (2).jpg'  # Your test image
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)



In [ ]:
# Generate CAM
targets = [ClassifierOutputTarget(1)]  # Assuming label 1 corresponds to 'infected'
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)


In [ ]:

# Visualize the result
grayscale_cam = grayscale_cam[0, :]
img_np = np.array(img) / 255.0  # Normalize image for visualization
cam_image = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)



In [ ]:
# Display the result
plt.imshow(cam_image)
plt.show()